In [1]:
import sympy as sym
import numpy as np
import time
import random
import copy

In [2]:
n = 2 # n*n node
coni = sym.MatrixSymbol("ci",n,n-1) #ตั้ง  ------
conj = sym.MatrixSymbol("cj",n-1,n) #นอน lllll

def get_squre_FEM(n,coni,conj):
    #check condition
    path = sym.zeros(n**2)   # 10x10 node = matrix [100x100]
    for i in range(n):
        for j in range(n):
            number_of_node = i+n*j
            Coefficient = 0
            if i != 0:
                #print(str(coni[j,i])+str(number_of_node))
               path[number_of_node,number_of_node] -= coni[j,i-1]
               path[number_of_node,number_of_node-1] += coni[j,i-1]
            if i != n-1:
              path[number_of_node,number_of_node] -= coni[j,i]
              path[number_of_node,number_of_node+1] += coni[j,i]
    for i in range(n):
        for j in range(n):
            number_of_node = i+n*j
            Coefficient = 0
            if j != 0:
                #print(str(coni[j,i])+str(number_of_node))
                path[number_of_node,number_of_node] -= conj[j-1,i]
                path[number_of_node,number_of_node-n] += conj[j-1,i]
            if j != n-1:
                path[number_of_node,number_of_node] -= conj[j,i]
                path[number_of_node,number_of_node+n] += conj[j,i]
    return(path)
    
def get_init_potential(n,coni,conj,currentInject,injectNode,groundnode):
    cai = sym.Matrix(np.round(np.random.rand(n, n-1),decimals = 2))
    caj = sym.Matrix(np.round(np.random.rand(n-1, n),decimals = 2))
    path = get_squre_FEM(n,coni,conj)
    path = path.subs({coni:cai,conj:caj})
    return(getNodepotential(path,currentInject,injectNode,groundnode))

def getNodepotential(path,currentInject,injectNode,groundnode):
    pathh = copy.deepcopy(path)
    size = sym.shape(pathh)[0]
    inject = sym.matrices.Matrix.zeros(size,1)
    inject[injectNode] = -currentInject
    pathh[groundnode,:] = [[0]*groundnode+[1]+[0]*(size-groundnode-1)] #standard unit column vector
    node_voltage = pathh.LUsolve(inject)
    return(node_voltage.evalf(3))
def getListCon(coni,conj):
    cli = []
    clj = []
    for i in range(n*(n-1)):
        cli += [coni[i]]
        clj += [conj[i]]
    return( sym.Matrix(cli+clj))
def getSurface(n):
    number = list(range(0, n**2))
    sur = number[0:n]+number[2*n-1:n**2:n]+number[n**2-n+1:n**2-1][::-1]+number[n:n**2:n][::-1]
    return sur
def getConfromList(con,n):
    cli = sym.Matrix(con[0:n*(n-1)]).reshape(n, n-1)
    clj = sym.Matrix(con[n*(n-1):]).reshape(n-1,n)
    return(cli,clj)



In [ ]:
surNode = getSurface(n)
path = get_squre_FEM(n,coni,conj)
jarcob = []
for I in range(len(surNode)):
    SurVoltage = sym.zeros(len(surNode),1)
    voltage = getNodepotential(path,1,I,3)
    ii = 0
    for J in surNode:
        SurVoltage[ii] = [voltage[J]]
        ii +=1
    jarcob += [SurVoltage.jacobian([getListCon(coni,conj)])]
    print(I)

In [4]:
'''ciref = sym.Matrix(np.round(np.random.rand(n, n-1),decimals = 2))*100
cjref = sym.Matrix(np.round(np.random.rand(n-1, n),decimals = 2))*100


cai = sym.Matrix(np.round(np.random.rand(n, n-1),decimals = 2))
caj = sym.Matrix(np.round(np.random.rand(n-1, n),decimals = 2))'''

ciref=sym.ones(n, n-1)*2
cai=sym.ones(n, n-1)

cjref=sym.ones(n-1, n)*2
caj=sym.ones(n-1, n)

conref = getListCon(ciref,cjref)
cona = getListCon(cai,caj)

surNode = getSurface(n)
path = get_squre_FEM(n,coni,conj)

for i in surNode:
    for g in surNode:
        if i!=g:
            SurVoltage = sym.zeros(len(surNode),1)
            voltage = getNodepotential(path,1,i,g)
            ii = 0
            for T in surNode:
                SurVoltage[ii] = [voltage[T]]
                ii +=1
            J = sym.Matrix(SurVoltage).jacobian([getListCon(coni,conj)]).subs({coni:cai,conj:caj})
            print(str(i) + str(g))

            for oo in range(5):
                Dvolt = getNodepotential(path.subs({coni:ciref,conj:cjref}),1,i,g)-getNodepotential(path.subs({coni:cai,conj:caj}),1,i,g)
                print(Dvolt)
                ii = 0
                DSurVolt = sym.zeros(len(surNode),1)
                for T in surNode:
                    DSurVolt[ii] = [Dvolt[T]]
                    ii +=1

                
                Dcona = (J.T*J).inv() * J.T * DSurVolt
                cona += Dcona
                cai,caj = getConfromList(cona,n)
print(conref)
print(cona)


01
Matrix([[-0.375], [0], [-0.250], [-0.125]])


NonInvertibleMatrixError: Matrix det == 0; not invertible.

In [9]:
'''ciref = sym.Matrix(np.round(np.random.rand(n, n-1),decimals = 2))*100
cjref = sym.Matrix(np.round(np.random.rand(n-1, n),decimals = 2))*100


cai = sym.Matrix(np.round(np.random.rand(n, n-1),decimals = 2))
caj = sym.Matrix(np.round(np.random.rand(n-1, n),decimals = 2))'''

ciref=sym.ones(n, n-1)*2
cai=sym.ones(n, n-1)

cjref=sym.ones(n-1, n)*2
caj=sym.ones(n-1, n)

conref = getListCon(ciref,cjref)
cona = getListCon(cai,caj)

surNode = getSurface(n)
pathh = get_squre_FEM(n,coni,conj)

for i in [0]:#surNode:
    for g in [1]: #surNode:
        if i!=g:
            SurVoltage = sym.zeros(len(surNode),1)
            voltage = getNodepotential(pathh,1,i,g)
            #pathh = get_squre_FEM(n,coni,conj)
            ii = 0
            for T in surNode:
                SurVoltage[ii] = [voltage[T]]
                ii +=1
            #print(pathh)
            J = sym.Matrix(SurVoltage).jacobian([getListCon(coni,conj)])
            print(str(i) + str(g))

            for oo in range(2):
                Dvolt = getNodepotential(pathh.subs({coni:ciref,conj:cjref}),1,i,g)-getNodepotential(pathh.subs({coni:cai,conj:caj}),1,i,g)
                J = J.subs({coni:cai,conj:caj})
                print(Dvolt)
                ii = 0
                DSurVolt = sym.zeros(len(surNode),1)
                for T in surNode:
                    DSurVolt[ii] = [Dvolt[T]]
                    ii +=1
                print((J.T*J).det() != 0)
                if (J.T*J).det() != 0:
                    Dcona = (J.T*J).inv() * J.T * DSurVolt
                    cona += Dcona
                    print("cona  :    "+str(cona))
                    cai,caj = getConfromList(cona,n)


01
Matrix([[-0.375], [0], [-0.250], [-0.125]])
False
Matrix([[-0.375], [0], [-0.250], [-0.125]])
False


In [17]:
cai

Matrix([
[1],
[1]])

In [198]:
for oo in range(5):
                Dvolt = getNodepotential(pathh.subs({coni:ciref,conj:cjref}),1,i,g)-getNodepotential(pathh.subs({coni:cai,conj:caj}),1,i,g)
                print(Dvolt)
                ii = 0
                DSurVolt = sym.zeros(len(surNode),1)
                for T in surNode:
                    DSurVolt[ii] = [Dvolt[T]]
                    ii +=1

                if (J.T*J).det() != 0:
                    Dcona = (J.T*J).inv() * J.T * DSurVolt
                    cona += Dcona
                    print("cona  :    "+str(cona))
                    cai,caj = getConfromList(cona,n)

Matrix([[0], [0], [0], [0], [0], [0], [0], [0], [0]])
cona  :    Matrix([[57.5], [4.52e+3], [-528.], [-2.16e+4], [-2.52e+3], [5.35e+5], [-12.5], [537.], [-369.], [568.], [1.43e+4], [2.05e+5]])
Matrix([[0], [0], [0], [0], [0], [0], [0], [0], [0]])
cona  :    Matrix([[57.5], [4.52e+3], [-528.], [-2.16e+4], [-2.52e+3], [5.35e+5], [-12.5], [537.], [-369.], [568.], [1.43e+4], [2.05e+5]])
Matrix([[0], [0], [0], [0], [0], [0], [0], [0], [0]])
cona  :    Matrix([[57.5], [4.52e+3], [-528.], [-2.16e+4], [-2.52e+3], [5.35e+5], [-12.5], [537.], [-369.], [568.], [1.43e+4], [2.05e+5]])
Matrix([[0], [0], [0], [0], [0], [0], [0], [0], [0]])
cona  :    Matrix([[57.5], [4.52e+3], [-528.], [-2.16e+4], [-2.52e+3], [5.35e+5], [-12.5], [537.], [-369.], [568.], [1.43e+4], [2.05e+5]])
Matrix([[0], [0], [0], [0], [0], [0], [0], [0], [0]])
cona  :    Matrix([[57.5], [4.52e+3], [-528.], [-2.16e+4], [-2.52e+3], [5.35e+5], [-12.5], [537.], [-369.], [568.], [1.43e+4], [2.05e+5]])


In [194]:
i = 0
g = 3
print(getNodepotential(pathh.subs({coni:cai,conj:caj}),1,i,g))
'''pathh = pathh.subs({coni:cai,conj:caj})
size = sym.shape(pathh)[0]
inject = sym.matrices.Matrix.zeros(size,1)
inject[0] = -1
pathh[8,:] = [[0]*8+[1]+[0]*(size-8-1)] #standard unit column vector
node_voltage = pathh.LUsolve(inject)
node_voltage'''



Matrix([[0.0330], [0.00843], [0.00917], [0], [-0.000483], [3.84e-5], [-1.62e-5], [-1.26e-5], [0]])


'pathh = pathh.subs({coni:cai,conj:caj})\nsize = sym.shape(pathh)[0]\ninject = sym.matrices.Matrix.zeros(size,1)\ninject[0] = -1\npathh[8,:] = [[0]*8+[1]+[0]*(size-8-1)] #standard unit column vector\nnode_voltage = pathh.LUsolve(inject)\nnode_voltage'

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 